# Seeds Dataset – Multiclass Classification with a Feed-Forward ANN



This notebook builds and evaluates a feed-forward artificial neural network (ANN) for multiclass classification on the UCI **Seeds** dataset.

We will:
1. Load and explore the data.
2. Preprocess (train–test split, scaling).
3. Design and train a Keras ANN with two hidden layers.
4. Compare two variants differing in a key hyperparameter (dropout rate).
5. Evaluate models on a test set (accuracy, precision, recall, F1-score).
6. Visualize a confusion matrix for the best model.


## University of Niagara Falls Canada
### DAMO640, Machine Learning, Sec 8
### Renan da Silva Sousa - NF1012001
### Professor: Dr. William Pourmajid

# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix
)

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

# For reproducibility
np.random.seed(42)
tf.random.set_seed(42)


ModuleNotFoundError: No module named 'tensorflow'